In [2]:
!pip install datasets
!pip install transformers
!pip install tokenizers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_syst

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
from transformers import AutoTokenizer, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

In [4]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Whitespace

from tokenizers import Tokenizer, models, trainers, pre_tokenizers, processors

In [5]:
from transformers import PreTrainedTokenizerFast

In [6]:
# Load AG_NEWS dataset using the datasets library
dataset = load_dataset("ag_news")
dataset_train = dataset["train"]
dataset_test = dataset["test"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [7]:
# Initialize a BPE tokenizer
tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))

# Set the pre-tokenizer to split on whitespace
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

# Set up the trainer with special tokens
trainer = trainers.BpeTrainer(
    vocab_size=30522,
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

# Define a generator to yield batches of text
def batch_iterator(batch_size=1000):
    for i in range(0, len(dataset_train), batch_size):
        yield dataset_train[i:i + batch_size]["text"]

# Train the tokenizer
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

# Set the post-processor to add special tokens
tokenizer.post_processor = processors.TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", tokenizer.token_to_id("[CLS]")),
        ("[SEP]", tokenizer.token_to_id("[SEP]")),
    ],
)
# Enable truncation and padding
tokenizer.enable_truncation(max_length=128)
tokenizer.enable_padding(pad_id=tokenizer.token_to_id("[PAD]"), pad_token="[PAD]", length=128)

In [8]:
tokenizer.save("custom_tokenizer.json")

In [9]:
fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file="custom_tokenizer.json")

In [10]:
def preprocess_data(examples):
    # Tokenize the text
    encodings = tokenizer.encode_batch(examples["text"])

    # Extract input IDs and attention masks
    input_ids = [encoding.ids for encoding in encodings]
    attention_masks = [encoding.attention_mask for encoding in encodings]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_masks,
        "label": examples["label"]
    }


In [11]:
# Apply preprocessing
ds_train = dataset_train.map(preprocess_data, batched=True)
ds_test = dataset_test.map(preprocess_data, batched=True)

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [12]:
ds_train[0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2,
 'input_ids': [1,
  1545,
  202,
  16,
  4838,
  34,
  699,
  2448,
  4265,
  95,
  2197,
  11,
  219,
  12,
  219,
  15,
  6456,
  15,
  15842,
  14,
  1545,
  1575,
  10,
  78,
  22169,
  58,
  1837,
  103,
  13167,
  15,
  29724,
  605,
  14,
  192,
  8962,
  4028,
  362,
  16,
  2,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1

In [13]:
# Define Basic Embedding Model
class BasicEmbeddingModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=3)
        self.fc = nn.Linear(embed_dim, num_classes)

    def forward(self, input_ids, attention_mask, labels=None):
        embedded = self.embedding(input_ids)
        embedded = torch.mean(embedded, dim=1)  # Averaging embeddings
        logits = self.fc(embedded)

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return {"loss": loss, "logits": logits} if loss is not None else {"logits": logits}

In [14]:
# Model setup
embed_dim = 100
num_classes = 4
vocab_size = 30522
model = BasicEmbeddingModel(vocab_size, embed_dim, num_classes)
# Show model summary
print(model)

BasicEmbeddingModel(
  (embedding): Embedding(30522, 100, padding_idx=3)
  (fc): Linear(in_features=100, out_features=4, bias=True)
)


In [15]:
# Define compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": acc, "f1": f1}

In [16]:
# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [19]:
# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_test,
    tokenizer=fast_tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-19-49bcaa4246ad>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [20]:
# Train model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.351100,1.349280,0.501711,0.502869
2,1.334900,1.335080,0.549474,0.549214
3,1.329100,1.329926,0.562500,0.561290


TrainOutput(global_step=11250, training_loss=1.3432329047309028, metrics={'train_runtime': 112.9874, 'train_samples_per_second': 3186.196, 'train_steps_per_second': 99.569, 'total_flos': 0.0, 'train_loss': 1.3432329047309028, 'epoch': 3.0})